In [16]:
import importlib.util
import csv
import sys
import os
from unittest.mock import patch
from QiskitPBT.coordinator import Coordinator

PATH = os.path.abspath("")

def import_function(module_str, path, function_name):    
    spec = importlib.util.spec_from_file_location(module_str, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_str] = module
    spec.loader.exec_module(module)    
    return getattr(module, function_name)

In [17]:
def run_tests(algorithm_name, num_inputs, measurements):
    results = []
    for i in range(0, 10):
        circuit_function = import_function(f"{algorithm_name}_m{i}", f"{PATH}\\case_studies\\{algorithm_name}\\mutants\\{algorithm_name}_m{i}.py", algorithm_name)
        with (patch(f"QiskitPBT.case_studies.{algorithm_name}.{algorithm_name}.{algorithm_name}", circuit_function)):
            coordinator = Coordinator(num_inputs, 1)
            result = coordinator.test(f"{PATH}\\case_studies\\{algorithm_name}", measurements) 
            
            # analyse results, add them to a list representing a csv file row
            num_circuits_executed = result.number_circuits_executed
            failed_properties = result.failed_property
            unique_properties = []
            failed_property_string = ""
            for property in failed_properties:
                if property.property.__class__ not in unique_properties:
                    failed_property_string += property.property.__class__.__name__ + " & "
                    unique_properties.append(property.property.__class__)
            failed_property_string = failed_property_string[:-3]
            num_unique_failed_properties = len(unique_properties)
            num_failed_properties = len(failed_properties)
            outcome = "Pass"
            if num_failed_properties > 0:
                outcome = "Fail"
                
            results.append([f"{algorithm_name}_m{i}", outcome, num_circuits_executed, num_unique_failed_properties, num_failed_properties, failed_property_string])
    # copy the for loop again for equivalent mutants, but change the name to f"{algorithm_name}_em{i}"
    return results

In [18]:
# I have written this specifically such that all the separate results of different configurations go in separate files 
# Just in case something goes horribly run when running, we won't lose everything
def test_and_store(algorithm_name):
    inputs = [3, 2, 1]
    shots = [15, 10, 5]
    for input in inputs:
        for measurements in shots:
            print(f"number of inputs: {input}, number of measurements: {measurements}")
            filename = f"mutation_test_results/{algorithm_name}/{algorithm_name}_{input}_{measurements}_mt_results.csv"
            dir = os.path.dirname(filename)
            if dir and not os.path.exists(dir):
                os.makedirs(dir)
            with open(filename, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(["Mutant Name", "Result", "Number of Circuits Executed", "Number of Unique Failed Properties", "Number of Failed Properties", "Unique Failed Properties",])
                csvwriter.writerows(run_tests(algorithm_name, input, measurements))
                csvfile.close()

In [19]:
test_and_store("quantum_teleportation")

number of inputs: 3, number of measurements: 15
{<class 'not_teleported_registers_equal_to_plus_property.NotTeleportedPlus'>, <class 'input_reg0_equal_to_output_reg2_property.Inq0EqualOutq2'>, <class 'unitary_before_teleport_equal_unitary_after_teleport_property.UnitaryBeforeAndAfterTeleport'>}
preflight steps 0.8461451530456543
circuit execution time 0.0955190658569336
measurement allocation time 0.03851175308227539
p val calc time 0.02599954605102539
{<class 'not_teleported_registers_equal_to_plus_property.NotTeleportedPlus'>, <class 'input_reg0_equal_to_output_reg2_property.Inq0EqualOutq2'>, <class 'unitary_before_teleport_equal_unitary_after_teleport_property.UnitaryBeforeAndAfterTeleport'>}
preflight steps 0.7621281147003174
circuit execution time 0.10851597785949707
measurement allocation time 0.03899812698364258
p val calc time 0.02852153778076172
{<class 'not_teleported_registers_equal_to_plus_property.NotTeleportedPlus'>, <class 'input_reg0_equal_to_output_reg2_property.Inq0Eq


KeyboardInterrupt



In [ ]:
test_and_store("deutsch_jozsa")

In [6]:
test_and_store("grovers_algorithm")

number of inputs: 3, number of measurements: 50
{<class 'most_frequent_output_should_not_be_marked_when_too_many_marks.GroversAlgorithmMostFrequentNotMarkedIfTooManyMarked'>, <class 'most_frequent_should_be_marked.GroversAlgorithmMostFrequentMarked'>, <class 'grover_lower_register_minus_property.GroversAlgorithmLowerRegisterMinus'>}
preflight steps 4.869844913482666
circuit execution time 0.7443442344665527
measurement allocation time 5.097204208374023
p val calc time 0.0075130462646484375
[['grovers_algorithm_m0'], 'Fail', 6, 'GroversAlgorithmLowerRegisterMinus & GroversAlgorithmLowerRegisterMinus & GroversAlgorithmLowerRegisterMinus & GroversAlgorithmMostFrequentMarked & GroversAlgorithmMostFrequentMarked & GroversAlgorithmMostFrequentMarked', 15]
{<class 'most_frequent_output_should_not_be_marked_when_too_many_marks.GroversAlgorithmMostFrequentNotMarkedIfTooManyMarked'>, <class 'most_frequent_should_be_marked.GroversAlgorithmMostFrequentMarked'>, <class 'grover_lower_register_minus_


KeyboardInterrupt



In [ ]:
test_and_store("quantum_fourier_transform")

In [ ]:
test_and_store("quantum_phase_estimation")